**UNIVERSIDAD NACIONAL DE SAN ANTONIO ABAD DEL CUSCO**
> **CURSO:** Minería de datos

> **ALUMNO:**Vega Centeno Olivera, Ronaldinho

# **5 Ejercicios de preprocesamiento**
Implementar 5 algoritmos a su elección de las diapositivas de pre-procesamiento en un notebook Spark.

## **Instalar bibliotecas**

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 49.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=5917d48be16f17c40886c417afad01b4623f4905c7253ed6bbed567112cb75f7
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


## **Conexion a cluster Spark (SparkContext)**

In [4]:
from pyspark import SparkContext
sc =SparkContext() #Permite la creación y manipulación de RDDs

## **1.Distancia de Minkowski con Spark**

In [48]:
def subtraction_p(tupla,p):
  """
    Calcula |xi-yi|^p
  """
  return abs(tupla[0]-tupla[1])**p

In [51]:
def distanceMinkowskiSpark(v1,v2,p,paticiones=4):
  """
  Calcula la distancia de Minkowski

  Parameters
  =========
    `v1, v2 :` vectores numericos 
          ej -> [1,7,8,9,2,15,14,6]
    `particiones :` valor numerico entero  
          ej -> 4

  Returns
  =========
    `distance :` valor numerico float
          ej -> 0.8

  """
  v3=list(zip( v1, v2))    #Creamos tuplas para cada i de los vectores (v1,v2)
  coordenadasRDD=sc.parallelize(v3,paticiones)  #Creamos el RDD
  #Calculamos todos los |xi-yi|^p
  subtraction_p_RDD=coordenadasRDD.map(lambda x: subtraction_p(x,p))
  sumRDD=subtraction_p_RDD.sum()                #calculamos la sumatoria
  distance=sumRDD**(1/p)                        #calculamos la raiz
  return distance

### **1.1. Ejemplos**

In [52]:
v1=list(range(1,5))
v2=list(range(4,8))
p=2
# Usando Spark 
print("La distancia de minkowski usando SPARK ES :", distanceMinkowskiSpark(v1,v2,p))
# Usando la libreria Scipy
from scipy import spatial
d=spatial.distance.minkowski(v1, v2, p=p, w=None)
print("La distancia de minkowski usando scipy ES :", d)


La distancia de minkowski usando SPARK ES : 6.0
La distancia de minkowski usando scipy ES : 6.0


## **2.Similitud de cosenos con Spark**

In [37]:
def internalProduct(tupla):
  """
    Calcula el producto
  """
  return tupla[0]*tupla[1]

In [43]:
def cosineSimilarity(v1,v2, particiones=4):
  """
    Calcula la similitud de cosenos entre 2 vectores

    Parameters
  =========
    `v1, v2 :` vectores numerico  
          ej -> [1,7,8,9,2,15,14,6]
    `particiones :` valor numerico entero  
          ej -> 4

  Returns
  =========
    `Similarity :` valor numerico float
          ej -> 0.8

  """
  #Creamos tuplas para cada i de los vectores (v1,v2)
  v3=list(zip( v1, v2))
  #creamos los RDD
  RDD_1=sc.parallelize(v1,particiones)
  RDD_2=sc.parallelize(v2,particiones)
  RDD_3=sc.parallelize(v3,particiones)
  #calculamos el producto interno
  sumInternalProductRDD=RDD_3.map(lambda x: internalProduct(x)).sum()
  #Calculamos la norma de v1
  normV1RDD= RDD_1.map(lambda x:x**2).sum()
  #Calculamos la norma de v2
  normV2RDD= RDD_2.map(lambda x:x**2).sum()
  #calculo final
  Similarity=(sumInternalProductRDD) / ( normV1RDD**(1/2) * normV2RDD**(1/2) )
  return Similarity

### **2.1. Ejemplos**

In [44]:
v1=list(range(1,5))
v2=list(range(4,8))
# Usando Spark 
print("La similitud de cosenos usando SPARK ES :", cosineSimilarity(v1,v2))
# Usando la libreria Scipy
from scipy import spatial
result = 1 - spatial.distance.cosine(v1, v2)
print("La similitud de cosenos usando scipy ES :", result)

La similitud de cosenos usando SPARK ES : 0.9759000729485332
La similitud de cosenos usando scipy ES : 0.9759000729485332


## **3.Normalización**

In [40]:
def normalize(vector, particiones=4):
  """
  Normaliza un Vector

  Parameters
  =========
    `vector :` vector numerico  
          ej -> [1,7,8,9,2,15,14,6]
    `particiones :` valor numerico entero  
          ej -> 4

  Returns
  =========
    `vector_normalizado :` vector numerico float
          ej -> [0.8, 0.6]

  """
  #creamos el RDD
  vectorRDD=sc.parallelize(vector,particiones)
  #calculamos el módulo del vector
  moduloVectorRDD= (vectorRDD.map(lambda x:x**2).sum())**(1/2)
  #normalizamos
  normalizeRDD=vectorRDD.map(lambda x : x/moduloVectorRDD )
  vector_normalizado=normalizeRDD.collect()
  return vector_normalizado

### **3.1.Ejemplo**

In [45]:
v1=range(1,5)
normalize(v1)

[0.18257418583505536,
 0.3651483716701107,
 0.5477225575051661,
 0.7302967433402214]

## **4.Escalonamiento**

In [21]:
def escalonamiento(vector, particiones=4):
  """
  Escalomaniento de un Vector

  Parameters
  =========
    `vector :` vector numerico  
          ej -> [1,7,8,9,2,15,14,6]
    `particiones :` valor numerico entero  
          ej -> 4

  Returns
  =========
    `vectorEscalonado :` vector numerico del tipo float
          ej -> [-1.0 , 0.4]

  """
  vectorRDD=sc.parallelize(vector,particiones)  #creamos el RDD
  minVector=vectorRDD.min()                     #hallamos el valor minimo
  maxVector=vectorRDD.max()                     #hallamos el valor máximo
  diferencia=maxVector-minVector                # calculamos la diferencia entre el max y min
  escalomaniemtoRDD= vectorRDD.map(lambda x : (x - maxVector)/ diferencia )
  vectorEscalonado=escalomaniemtoRDD.collect()  #recuperamos los valores
  return vectorEscalonado 

### **4.1.Ejemplo**

In [46]:
v1 = range(1,5)
escalonamiento(v1,4)

[-1.0, -0.6666666666666666, -0.3333333333333333, 0.0]

## **5.Estandarización**


In [26]:
def estandarizar(vector, particiones=4):
  """
  Estandarizacion de un Vector

  Parameters
  =========
    `vector :` vector numerico  
          ej -> [1,7,8,9,2,15,14,6]
    `particiones :` valor numerico entero  
          ej -> 4

  Returns
  =========
    `vectorEstandarizado :` vector numerico del tipo float
          ej -> [-1.0 , 0.4]

  """
  vectorRDD=sc.parallelize(vector,particiones)  #creamos el RDD
  meanVector=vectorRDD.mean()                   #hallamos la media del vector
  dsVector=vectorRDD.stdev()                   #hallamos la desviacion estandar
  estandarizacionRDD= vectorRDD.map(lambda x : (x - meanVector)/ dsVector )
  vectorEstandarizado=estandarizacionRDD.collect()  #recuperamos los valores
  return vectorEstandarizado 

### **5.1.Ejemplo**

In [47]:
v1 = range(1,5)
estandarizar(v1,4)

[-1.3416407864998738,
 -0.4472135954999579,
 0.4472135954999579,
 1.3416407864998738]